# Project 3

### Task 1

Members: Lars Olav Thorbjørnsen, Stein Are Årsnes og Sanjai Vijayaratnam

### Abstract

Data preprocessing and feature selection are essential steps in machine learning(ML) to ensure model accuracy and reliability. In this task, we take the raw log data, containing shear wave velocity (Vs), density (DEN), neutron porosity (NEU) and compressional wave velocity (Vp), clean it and prepare it for analysis. We start with checking that all data is of the float type before moving on to missing values, duplicates, outliers and conducting a correlation analysis. After removing missing values, duplicates and most of the outliers, we choose too keep all features since they are all above the 0.5 threshold. In the end we do some filtering and display the results. We have now laid a strong foundation for ML modeling, improving the reliability of Vp estimation from the log data.

### Introduction

Effective data preprocessing and feature selection are important in building accurate ML models. In this project, raw data logs with attributes like Vs, DEN and NEU needs to be cleaned and prepared before being used to estimate Vp. Raw data could be incomplete, redundant or noisy, by data preprocessing these issues can be fixed [1]. This task aimed to enhance data quality and select meaningful predictors to ensure the effectiveness of ML models.

### Task 1: Data Pre-processing

In [15]:
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
df = pd.read_excel('ProjectData2024.xlsx')
df.info()
df.head(), df.tail(20)



We start of with importing the libraries we need for this task and then import the data file. Now that we have the data file we take a quick look at it and discover that the file has many missing values. We also see that all the values are of type float so any converting will not be needed. 

In [16]:
df.replace(0, np.nan, inplace=True)
df.replace('', np.nan,inplace=True)
df.info()
df.head(), df.tail(20)

df.dropna(inplace=True) #Removes/drops all Nan values in df
miss_data= df.isnull().sum() #checks if there is any missing values
print(miss_data)
df.info()

The method we have used to remove missing values is to replace missing or empty values with NaN and then removing all NaN values. As you can see we now have 0 missing values. We move on to duplicates.

In [17]:
dups= df.duplicated()
print(dups.any())
print(df[dups])


We check if the data frame contains any duplicates, it does as the dups.any() function returns true. We then print set duplicates and discover quite allot of values. Now starts the process of removing them.

In [18]:
df.drop_duplicates(inplace = True)

dups2= df.duplicated()
print(dups2.any())

df.info()

After removing the duplicates we again check if there are any and this time it returns false, we have removed all duplicates. We move on to outliers.

In [19]:
df.shape
plt.boxplot(df,widths=0.5)

By plotting this boxplot we clearly sees that there is many outliers in our data frame. We can also look at a single column.

In [20]:
def plot_boxplot(df, ft):
    df.boxplot(column=[ft])
    plt.grid(False)
    plt.show()
plot_boxplot(df, "Vp")

We create a function for plotting a boxplot of a specific column in the data frame. As shown in the boxplot we see many outliers in the Vp column. Now starts the task of removing set outliers.

In [21]:
def outliers(df, ft):
    Q1 = df[ft].quantile(0.25)
    Q3 = df[ft].quantile(0.75)
    IQR = Q3 - Q1
    UB = Q3 + 1.5*IQR
    LB = Q1-1.5*IQR
    ls = df.index[ (df[ft] < LB) | (df[ft] > UB)]
    return ls

index_list = []
for f in ['Vs', 'DEN', 'NEU', 'Vp']:
    index_list.extend(outliers(df, f))

def remove(df, ls):
    ls = sorted(set(ls))
    df = df.drop(ls)
    return df

df_cleaned = remove(df, index_list)

print("New shape", df_cleaned.shape)
df_cleaned.info()
plt.boxplot(df_cleaned,widths=0.5)
df_cleaned.to_excel('CleanOutlier1.xlsx', index=False)

We create a function for locating all outliers one column at a time using the IQR method. Then we create a for loop that loops for all columns and inserts the outliers for each column into a list. We also create a function that takes a list and a data frame, this function we use to remove all outliers from our data frame using the list of outliers. We now see that almost all of the outliers are removed.

In [22]:
plot_boxplot(df_cleaned, "Vp")

This is how the Vp column now looks after removing outliers. Much better than before. We will now look at correlation.

In [23]:
 
df2 = pd.read_excel('CleanOutlier1.xlsx').astype(float)
print(df2.head()) 
sns.heatmap(df2.corr(), annot = True, fmt= '0.4')
CorrelationData=df2.corr()

CorrelationData

We display the correlation data in numbers and in a heat map, from this data we see that Vs, DEN and NEU all have a big correlation with Vp. Both Vs and DEN have a positive correlation of almost the exact same size, NEU has a negative correlation which means that when one gets larger the other grows smaller [2]. NEUs correlation is the greatest. 

In [24]:
CorrelationData['Vp'][abs(CorrelationData['Vp']) > 0.5]


According to the project 0.5 is a good threshold to use when selecting features, we see that all our features pass this threshold so we will not drop any features.

In [25]:
df3 = pd.read_excel('CleanOutlier1.xlsx')
df3.info()
plt.plot(df3)
plt.ylim(0,5)


We plot the cleaned data and we see that the data contains some noise, to get clearer data we can apply some filtering/smoothening. 

In [26]:
plt.plot(df3['Vp'])
dfx1 = df3['Vp'].rolling(window =15).mean().plot(figsize=(10,6))
plt.show()

plt.plot(df3['Vs'])
dfx1 = df3['Vs'].rolling(window =15).mean().plot(figsize=(10,6))
plt.show()

plt.plot(df3['DEN'])
dfx1 = df3['DEN'].rolling(window =15).mean().plot(figsize=(10,6))
plt.show()

plt.plot(df3['NEU'])
dfx1 = df3['NEU'].rolling(window =15).mean().plot(figsize=(10,6))
plt.show()

dataF = df3.rolling(window =15).mean()
dataF.dropna(inplace=True) #Some values went missing after filtering


The graphs show the data with and without the filter, orange being with filter. We see that there is less noise/spikes and the data is more consistent.  

In [27]:
plt.plot(dataF)
plt.ylim(0,5)
plt.show()
dataF.to_excel('CleanedFeatureSelectedFiltered.xlsx', index=False)

Again the graph with all the data now shows much less spikes/noise. 

### Conclusion 
The data preprocessing and feature selection steps were key in enhancing the data quality and model readiness. We also saw how big a difference filtering made in terms of noise and spikes. By handling data inconsistencies and focusing on high-correlation features we have improved the accuracy potential of our Vp estimation models. This shows how important data preprocessing is to create a reliable foundation for the ML models to follow. 

### Reflection
Reflecting over this data preprocessing task, we got a new understanding of how important data integrity is for ML projects. We also learned about how correlation works in terms of feature selection with both negative and positive correlation. Another observation is how big a difference filtering makes, just looking at graphs really shows how important filtering can be when reducing noise.

### References
[1]:Amit Kumar Tyagi, Ajith Abraham,"2.5.1 Data preprocessing", Data science for Genomics, 2023


[2]:JOVE 1.13 Correlations, 09.11.2024, https://www.jove.com/science-education/11030/correlation-correlation-coefficient-positive-negative-correlation 